# PPDB HATS

### Demo and data validation

In this notebook we will:

1. Simulate two PPDB daily increments.
3. Perform a full collection reimport.
4. Validate the data: the deduplication of objects and aggregation of sources.

Ran on USDF, 2/19/2026.

In [ ]:
import subprocess
import lsdb
import nested_pandas as npd
import numpy as np
import pandas as pd

from datetime import date
from glob import glob
from pathlib import Path

from ppdb_hats.config import get_default_config

# Input directory for PPDB parquet files
PPDB_LSST_DIR = Path("/sdf/scratch/rubin/ppdb/data/ppdb_lsstcam")

# Output directory for PPDB HATS catalogs
PPDB_HATS_DIR = Path("/sdf/data/rubin/shared/lsdb_commissioning/ppdb")

### Utilities

In [ ]:
def find_files_for_date(date: date):
    """Find input parquet for DIA object, source and forced source"""
    date = date.strftime("%Y/%m/%d")
    obj_files = glob(f"{PPDB_LSST_DIR}/{date}/**/DiaObject*.parquet")
    print(f"Number of DiaObject files: {len(obj_files)}")
    src_files = glob(f"{PPDB_LSST_DIR}/{date}/**/DiaSource*.parquet")
    print(f"Number of DiaSource files: {len(src_files)}")
    forced_src_files = glob(f"{PPDB_LSST_DIR}/{date}/**/DiaForcedSource*.parquet")
    print(f"Number of DiaForcedSource files: {len(forced_src_files)}")
    return obj_files, src_files, forced_src_files

def get_pipeline_config(**kwargs):
    """Build custom pipeline config"""
    return get_default_config(**kwargs)

### Pre-existing catalog collection

Let's assume we already have a base collection, imported with Npix directories.

In [ ]:
collection_dir = PPDB_HATS_DIR / "dia_object_collection"
collection = lsdb.open_catalog(collection_dir)
assert collection.hc_structure.catalog_info.npix_suffix == "/"
print(f"Collection has {len(collection):,} points")
collection.plot_pixels()

It has data from Sept 6, 2025 to Jan 25, 2026. All the input parquet paths are stored under `input_paths`:

In [ ]:
subprocess.run(["head", "-1", f"{collection_dir}/input_paths/dia_object.txt"])
subprocess.run(["tail", "-1", f"{collection_dir}/input_paths/dia_object.txt"])

### Daily increment

We will increment with data from a couple of days (Jan 26 and Jan 28), one at a time.

In [ ]:
from ppdb_hats import DailyPipeline
config = get_pipeline_config(until_date=date(2026, 1, 26))
DailyPipeline(config=config).execute()

#### Validation of first increment

Let's validate that the data looks consistent.

In [ ]:
obj_files, src_files, forced_src_files = find_files_for_date(date(2026,1,26))

The `input_paths` have been updated:

In [ ]:
subprocess.run(["tail", "-6", f"{collection_dir}/input_paths/dia_object.txt"])

In [ ]:
increment = lsdb.open_catalog(collection_dir)
print(f"The new collection has {len(increment):,} objects.")
print(f"The increment added {len(increment) - len(collection):,} new objects.")

All the new objects exist in the new collection:

In [ ]:
new_objects = npd.read_parquet(obj_files)
unique_ids = new_objects["diaObjectId"].unique()
increment_ids = increment["diaObjectId"].compute()
assert set(unique_ids).issubset(set(increment_ids))

Let's look at a specific object (selected to have both sources and forced sources):

In [ ]:
my_obj = increment.query(f"diaObjectId == 169738311479853178")
my_obj.compute()

The first entry is new PPDB data, with information on `i-band`:

In [ ]:
# Looking at the original input object parquet
new_objects.query(f"diaObjectId == 169738311479853178")

Making sure this object's new sources were appended correctly:

In [ ]:
my_obj["diaSource"].compute().explode()

In [ ]:
# Looking at the original input source parquet
npd.read_parquet(src_files).query(f"diaObjectId == 169738311479853178")

We keep track of the already imported parquet paths. On an increment, those are ignored.

#### Validation of second increment

In [ ]:
obj_files, src_files, forced_src_files = find_files_for_date(date(2026, 1, 28))

The pipeline will succeed when there's no new data for forced sources:

In [ ]:
config = get_pipeline_config(until_date=date(2026, 1, 28))
DailyPipeline(config=config).execute()

In [ ]:
increment2 = lsdb.open_catalog(collection_dir)
print(f"The collection has {len(increment2):,} objects.")
print(f"The increment added {len(increment2) - len(increment):,} new objects.")

We again did not lose data in this increment:

In [ ]:
new_objects = npd.read_parquet(obj_files)
unique_ids = new_objects["diaObjectId"].unique()
increment2_ids = increment2["diaObjectId"].compute()
assert set(unique_ids).issubset(set(increment2_ids))

Let's check an object which was seen in increment one and two:

In [ ]:
my_obj = increment2.query(f"diaObjectId == 169747015140900902").compute()
my_obj.sort_values("validityStartMjdTai")

We appended the most recent object data from the input files:

In [ ]:
new_objects.query(f"diaObjectId == 169747015140900902").sort_values("validityStartMjdTai")

After each increment we have a parquet file for each date, which we can query (even with other parquet readers) to get specific date updates:

In [ ]:
subprocess.run(["ls", f"{collection_dir}/dia_object_lc/dataset/Norder=2/Dir=0/Npix=106"])

### Weekly reimport

This reprocessing step creates a weekly "release" of the collection, with new margins and indices.

In [ ]:
from ppdb_hats import WeeklyPipeline
WeeklyPipeline().execute()

We have slightly less rows because we de-duplicated object data:

In [ ]:
weekly_id = date.today().strftime("%Y%m%d")
weekly_dir = PPDB_HATS_DIR / "weekly" / f"dia_object_collection_{weekly_id}"
weekly = lsdb.open_catalog(weekly_dir)
print(f"Weekly has ~{len(weekly) / len(increment2) * 100:.2f}% of incremented rows")

The IDs match for both collections:

In [ ]:
weekly_ids = weekly["diaObjectId"].compute()
increment2_ids = increment2["diaObjectId"].compute()
unique_increment2_ids = np.unique(increment2_ids)
assert len(weekly_ids) == len(unique_increment2_ids)
assert set(weekly_ids) == set(unique_increment2_ids)

Let's grab our first object and make sure that its sources were correctly merged:

In [ ]:
weekly_obj = weekly.id_search({"diaObjectId": 169738311479853178}).compute()
weekly_obj

In [ ]:
# Checking sources
increm_obj_source = increment2.query("diaObjectId == 169738311479853178")["diaSource"]
expected_obj_source = increm_obj_source.explode().sort_values("midpointMjdTai").compute()
pd.testing.assert_frame_equal(expected_obj_source, weekly_obj["diaSource"].explode())

In [ ]:
# Checking forced sources
increm_obj_source = increment2.query("diaObjectId == 169738311479853178")["diaForcedSource"]
expected_obj_source = increm_obj_source.explode().sort_values("midpointMjdTai").compute()
pd.testing.assert_frame_equal(expected_obj_source, weekly_obj["diaForcedSource"].explode())